In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
import glob

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
os.chdir(r'/content/drive/Production')
extension = 'csv'
all_files = [i for i in glob.glob('*.{}'.format(extension))]
Prod = pd.concat([pd.read_csv(f
                             ,usecols=['Encounter #','DOS','Bucket','Practice','Financial Class','Payer','Disposition Code','Action Code','Activity Date','Associate',
                                       'Supervisor','Manager','Activity Note','Inventory Type','Source Of Action','Action','Created By','Billed Amount','Total Balance',]
                            ,low_memory=False, encoding='latin1') for f in all_files ])

In [6]:
os.chdir(r'/content/drive/Adjustment')
extension = 'csv'
all_files = [i for i in glob.glob('*.{}'.format(extension))]
Adj = pd.concat([pd.read_csv(f
                             ,usecols=['Ins_Pat_Flag','ClaimNumber','Post_Date','Adjustment']
                             ,low_memory=False) for f in all_files ])

In [7]:
os.chdir(r'/content/drive/Payments')
extension = 'csv'
all_files = [i for i in glob.glob('*.{}'.format(extension))]

Pay = pd.concat([pd.read_csv(f
                             ,usecols=['Ins_Pat_Flag','ClaimNumber','Post_Date','Payment']
                             ,low_memory=False) for f in all_files ])

In [8]:
##Column rename as per the standard script
Pay.rename(columns={'ClaimNumber':'E/I/A/B','Post_Date':'Post Date','Payment':'Pay Amt','Ins_Pat_Flag':'Source'},inplace=True)
Adj.rename(columns={'ClaimNumber':'E/I/A/B','Post_Date':'Post Date','Adjustment':'Adj Amt','Ins_Pat_Flag':'Source'},inplace=True)

Pay['Prac Name'] = np.nan
Adj['Prac Name'] = np.nan

#### Payment data preparation

In [9]:
df_Pay = Pay[Pay['Source']=='Insurance'][['Prac Name','E/I/A/B','Post Date','Pay Amt']] #This might change as per the EPMs
df_Pay['Post Date'] = pd.to_datetime(df_Pay['Post Date'], format='mixed').dt.normalize()


df_Pay['Pay Amt'] = df_Pay['Pay Amt'].astype(str)+"$"
df_Pay['Pay Amt'] = df_Pay['Pay Amt'].str.replace('$', "", regex=False)\
    .str.replace('(', "-", regex=False).str.replace(')', "", regex=False)\
        .str.replace(',', "", regex=False).str.replace(' ', "", regex=False)
df_Pay['Pay Amt']=df_Pay['Pay Amt'].astype(float)

df_Pay['Pay_Year'] = df_Pay['Post Date'].dt.year

df_Pay['Pay_Month'] = df_Pay['Post Date'].dt.month
df_Pay = df_Pay[['Prac Name','E/I/A/B','Pay Amt','Post Date','Pay_Year','Pay_Month']]
df_Pay = df_Pay.pivot_table(index=['E/I/A/B','Post Date','Pay_Year','Pay_Month'],
                            values='Pay Amt', aggfunc='sum').reset_index()
df_Pay['E/I/A/B']  =df_Pay['E/I/A/B'].map(str)
df_Pay['Enc'] = df_Pay['E/I/A/B'].str.split('.',expand=True)[0]
df_Pay['Enc'] =  df_Pay['Enc'].map(str)
df_Pay1=df_Pay[df_Pay['Pay Amt']>0]

#### Adjustment data preparation

In [10]:
df_Adj = Adj[Adj['Source']=='Insurance'][['Prac Name','E/I/A/B','Post Date','Adj Amt']]
df_Adj['Post Date'] = pd.to_datetime(df_Adj['Post Date'], format='mixed').dt.normalize()

df_Adj['Adj Amt']=df_Adj['Adj Amt'].astype(str)+"$"
df_Adj['Adj Amt'] = df_Adj['Adj Amt'].str.replace('$', "", regex=False)\
    .str.replace('(', "-", regex=False).str.replace(')', "", regex=False)\
        .str.replace(',', "", regex=False).str.replace(' ', "", regex=False)
df_Adj['Adj Amt']=df_Adj['Adj Amt'].astype(float)
df_Adj['Adj_Year'] = df_Adj['Post Date'].dt.year
df_Adj['Adj_Month'] = df_Adj['Post Date'].dt.month
df_Adj = df_Adj[['Prac Name','E/I/A/B','Adj Amt','Post Date','Adj_Year','Adj_Month']]
df_Adj = df_Adj.pivot_table(index=['E/I/A/B','Post Date','Adj_Year','Adj_Month'],
                            values='Adj Amt', aggfunc='sum').reset_index()
df_Adj['E/I/A/B']  =df_Adj['E/I/A/B'].map(str)
df_Adj['Enc'] = df_Adj['E/I/A/B'].str.split('.',expand=True)[0]
df_Adj['Enc'] =  df_Adj['Enc'].map(str)

df_Adj1=df_Adj[df_Adj['Adj Amt']>0]

#### Production data preparation

In [11]:
Prod['Practice_Code']=Prod['Encounter #'].str.split('-',expand=True)[0]
Prod['E/I/A/B']=Prod['Encounter #'].str.split('-',expand=True)[1]
df_Prod = Prod[(Prod['Practice_Code']=='2')]

df_Prod['Index_val'] = np.arange(1,len(df_Prod)+1, 1)

df_Prod['Activity Date'] = pd.to_datetime(df_Prod['Activity Date'], format = 'mixed').dt.normalize()
df_Prod.sort_values(by=['E/I/A/B','Activity Date'], ascending=True, inplace=True )
df_Prod['Prod_Year'] = df_Prod['Activity Date'].dt.year
df_Prod['Prod_Month'] = df_Prod['Activity Date'].dt.month

df_Prod['Total_Touch'] = df_Prod.groupby(['E/I/A/B']).cumcount()+1

In [12]:
df_Prod['Total Balance'] = df_Prod["Total Balance"].astype(str)+"$"
df_Prod['Total Balance']=df_Prod['Total Balance'].str.replace('$', "", regex=False)\
    .str.replace('(', "-", regex=False).str.replace(')', "", regex=False)\
        .str.replace(',', "", regex=False).str.replace(' ', "", regex=False)
df_Prod['Total Balance'].astype(float)
df_Prod['Billed Amount'] =df_Prod['Billed Amount'].astype(str)+"$"
df_Prod['Billed Amount'] =df_Prod['Billed Amount'].str.replace('$', "", regex=False)\
    .str.replace('(', "-", regex=False).str.replace(')', "", regex=False)\
        .str.replace(',', "", regex=False).str.replace(' ', "", regex=False)
df_Prod['Billed Amount'] = df_Prod['Billed Amount'].astype(float)

In [13]:
df_Prod['Enc'] = df_Prod['E/I/A/B'].map(str)
df_Prod['Enc'] = df_Prod['Enc'].str.split('.',expand=True)[0]
df_Prod['Enc'] =  df_Prod['Enc'].map(str)
df_Prod['Next_Activity_raw'] = df_Prod['Activity Date'].shift(-1)
df_Prod['Next_Enc'] = df_Prod['Enc'].shift(-1)

df_Prod['Next_Activity'] = df_Prod.apply(lambda x : x['Next_Activity_raw'] if x['Enc'] == x['Next_Enc'] else np.nan, axis=1)

df_Prod.drop(columns=['Next_Activity_raw','Next_Enc'], inplace=True)
df_Prod['Next_Activity'].fillna(df_Prod['Activity Date'], inplace=True)

### Mapping of Production with Payment

In [14]:
df_Prod_Merged = df_Prod.merge(df_Pay1, how='left', on = 'Enc')
df_Prod_Merged['Days_Diff'] = (df_Prod_Merged['Post Date'] - df_Prod_Merged['Activity Date']).dt.days
df_Prod_Merged['E/I/A/B_x'] = df_Prod_Merged['E/I/A/B_x'].astype(str)

df_Prod_Payment =  df_Prod_Merged[df_Prod_Merged['Post Date'].notna()]
df_Prod_Payment['Post_to_Activity'] = (df_Prod_Payment['Post Date'] - df_Prod_Payment['Activity Date']).dt.days
df_Prod_Payment['Post_to_Next_Activity'] = (df_Prod_Payment['Post Date'] - df_Prod_Payment['Next_Activity'] ).dt.days

In [15]:
def Valid_Payment_Ind(Post_Activity, Post_to_Next_Activity):

    Post_Activity = int(Post_Activity)
    Post_to_Next_Activity = int(Post_to_Next_Activity)
    output = ''

    if Post_Activity >=0 &  Post_to_Next_Activity >=0:
        if Post_Activity == Post_to_Next_Activity:
            output= 'Y'
        elif Post_Activity != Post_to_Next_Activity:
            output = 'N'
    elif Post_Activity >=0 & Post_to_Next_Activity <0:
        output= 'Y'
    elif Post_Activity <0 & Post_to_Next_Activity <0:
        output= 'N'
    else:
        output= 'N'

    return output
    #print(Post_Activity, Post_to_Next_Activity,output )
df_Prod_Payment['Valid_Payment'] = df_Prod_Payment.apply(lambda x : Valid_Payment_Ind(x['Post_to_Activity'],
                                                                                         x['Post_to_Next_Activity']),
                                                           axis=1)

### Mapping of Production with Adj

In [16]:
df_Prod_Merged_Adj = df_Prod.merge(df_Adj1, how='left', on = 'Enc')
df_Prod_Merged_Adj['Days_Diff'] = (df_Prod_Merged_Adj['Post Date'] - df_Prod_Merged_Adj['Activity Date']).dt.days
df_Prod_Merged_Adj['E/I/A/B_x'] = df_Prod_Merged_Adj['E/I/A/B_x'].astype(str)

df_Prod_Adj =  df_Prod_Merged_Adj[df_Prod_Merged_Adj['Post Date'].notna()]
df_Prod_Adj['Post_to_Activity'] = (df_Prod_Adj['Post Date'] - df_Prod_Adj['Activity Date']).dt.days
df_Prod_Adj['Post_to_Next_Activity'] = (df_Prod_Adj['Post Date'] - df_Prod_Adj['Next_Activity'] ).dt.days
df_Prod_Adj['Valid_Adj'] = df_Prod_Adj.apply(lambda x : Valid_Payment_Ind(x['Post_to_Activity'],
                                                                                         x['Post_to_Next_Activity']),
                                                           axis=1)

In [17]:
df_Prod_Adj_F= df_Prod_Adj[df_Prod_Adj['Valid_Adj']=='Y'][['Index_val','Post Date','Adj Amt']]
df_Prod_Pay_F= df_Prod_Payment[df_Prod_Payment['Valid_Payment']=='Y'][['Index_val','Post Date','Pay Amt']]


### Source DFs df_Prod_Adj_F, df_Prod_Pay_F
#### New DF from df_Prod, merged with above DFs to tag the payments and Adjustments

In [18]:
# Join the production with Adjustment and update liquidation month
df_Prod_Adj_f1 = df_Prod.merge(df_Prod_Adj_F, how='inner', on='Index_val')
df_Prod_Adj_f1['Date_Diff'] = np.round((((df_Prod_Adj_f1['Post Date'] - df_Prod_Adj_f1['Activity Date']).dt.days)/30),0)

# Join the production with Payment and update liquidation month
df_Prod_Pay_f1 = df_Prod.merge(df_Prod_Pay_F, how='inner', on='Index_val')
df_Prod_Pay_f1['Date_Diff'] = np.round((((df_Prod_Pay_f1['Post Date'] - df_Prod_Pay_f1['Activity Date']).dt.days)/30),0)

df_Prod_Adj_f2 = df_Prod_Adj_f1.groupby(by=['Index_val','Date_Diff'])['Adj Amt'].sum().reset_index()#reset_index(name='Amount')
df_Prod_Adj_f2['Liq_Category'] = 'Adj'
df_Prod_Pay_f2 = df_Prod_Pay_f1.groupby(by=['Index_val','Date_Diff'])['Pay Amt'].sum().reset_index()#(name='Amount')
df_Prod_Pay_f2['Liq_Category'] = 'Pay'

df_Prod_liq_Final = pd.concat([df_Prod_Adj_f2,df_Prod_Pay_f2])
df_Prod_final_1 = df_Prod.merge(df_Prod_liq_Final, how='left', on='Index_val')

# Add index column to identify the duplicates
df_Prod_final_1['Index'] = np.arange(1,len(df_Prod_final_1)+1,1)
unique_index = df_Prod_final_1.groupby(by=['Index_val'])['Index'].min().reset_index(name='Index')

df_Prod_final_1.set_index('Index', inplace=True)

In [19]:
df_Prod_final_1['Encounter'] = np.nan
df_Prod_final_1['Balance_worked'] = 0.0
df_Prod_final_1['Resolved_Encounter']  = np.nan

df_Prod_final_1.loc[unique_index['Index'].unique(),'Encounter'] = df_Prod_final_1['Enc']
df_Prod_final_1.loc[unique_index['Index'].unique(),'Balance_worked'] = df_Prod_final_1['Total Balance']

df_Prod_final_1['Resolved']  =df_Prod_final_1['Liq_Category'].apply(lambda x : 'Y' if pd.notnull(x) else 'N')
df_Prod_final_1['Resolved_Encounter']  =df_Prod_final_1.apply(lambda x : x.Enc if x.Resolved == 'Y' else '', axis=1)

In [20]:
df_Prod_final_2 = df_Prod_final_1
df_Prod_final_2.reset_index(inplace=True)
df_Liq_cat = df_Prod_final_2.pivot_table(index='E/I/A/B', columns='Liq_Category',values='Resolved_Encounter', aggfunc='count').reset_index()
df_Liq_cat = df_Liq_cat[df_Liq_cat['Pay'].isna()]
df_Liq_cat_Encs = df_Liq_cat['E/I/A/B']
Full_Adjusted_indx = df_Prod_final_1[df_Prod_final_1['E/I/A/B'].isin(df_Liq_cat_Encs)].index
df_Prod_final_1['Fully_Adjusted'] = np.nan
df_Prod_final_1.loc[Full_Adjusted_indx,'Fully_Adjusted'] = df_Prod_final_1['Enc']

In [21]:
Desp = pd.read_csv(r'/content/drive/disp.csv')
df_Prod_final_1 = df_Prod_final_1.merge(Desp, on='Disposition Code', how='left')
df_Prod_final_1['Associate']=df_Prod_final_1['Associate'].str.split("(", expand = True)[0]
df_Prod_final_1['Supervisor']=df_Prod_final_1['Supervisor'].str.split("(", expand = True)[0]
df_Prod_final_1['Activity_Year'] = df_Prod_final_1['Activity Date'].dt.year
df_Prod_final_1['Activity_Month']=df_Prod_final_1['Activity Date'].dt.month
df_Prod_final_1['Prac_Type'] = df_Prod_final_1['Practice'].apply(lambda x : 'Lab' if x in ['Mid-Atlantic POL','Mid-West POL'] else 'Non Lab')

In [22]:
def inv(rows):
    action_code = rows['Action Code']

    if 'Appeal' in action_code or \
       'Investigation' in action_code or \
       'Redetermination' in action_code:
        val = "Appeal"
    #elif rows['Bucket'] == 'ERA Payer Denial':
        #val = "Denial"
    else:
        val = "Follow up"

    return val

df_Prod_final_1['Inventory_type'] = df_Prod_final_1.apply(inv, axis=1)

In [23]:
Action_CD = ["CLIENT ASSIST-CODING-CLIENT ASSIST-CODING","CLIENT ASSIST-FMC-CLIENT ASSIST-FMC",
             'CLIENT ASSIST-NEED MR-CLIENT ASSIST-NEED MR',"CLIENT ASSIST - AUTH/REF-CLIENT ASSIST - AUTH/REF",
             "CLIENT ASSIST-DEMO MISMCH-CLIENT ASSIST-DEMO MISMCH","CLIENT ASSIST - CREDENTIALING-CLIENT ASSIST - CREDENTIALING",
             "CLIENT ASSIST-INFO INS-CLIENT ASSIST-INFO INS","CLIENT ASSIST-CHG CORRECT-CLIENT ASSIST-CHG CORRECT",
             "CLIENT ASSIST-REJECTIONS-CLIENT ASSIST-REJECTIONS","CLIENT ASSIST-LITIGATION-CLIENT ASSIST-LITIGATION",
             "CLIENT ASSIST - CREDENTIALING-CLIENT ASSIST - CRED","CLIENT ASSIST-DEMO MISMCH-CLIENT ASSIST-DEMO MISMC"
             "CLAIM PRCOCESSED - PAYMENT PENDING-CLAIM PRCOCESSED - PAYMENT PENDING",
             "CLAIM PRCOCESSED - PAYMENT PENDING-CLAIM PRCOCESSE","PYMT POSTING-PEND>10DAYS-PYMT POSTING-PEND>10DAYS",
             "CHK CASHED REQSTD CHK CPY-CHK CASHED REQSTD CHK CPY","Adj > 500-Adj > 500","BILLED PATIENT-BILLED PATIENT",
             "FWD to IKS Supervisor-FWD to IKS Supervisor","IKS - Billing Assistance-IKS - Billing Assistance",
             "PAR ASSISTANCE-PAR ASSISTANCE","Payment Posting-Payment Posting",
             "Pending for approval-Pending for approval","RECENTLY PD < 10 DAYS-RECENTLY PD < 10 DAYS",
             "REQUEST REFUND/OFFSET LETTER-REQUEST REFUND/OFFSET LETTER"]

def T (rows):

    if (rows['Disposition Code']=="Adj approval Pending-Adj approval Pending") or\
        (rows['Action Code']=="Adj < 500-Adj < 500"):
        val = 'Tasked'
    elif rows["Action Code"] in (Action_CD):
        val = 'Tasked'
    else:
        val = "Not Tasked"
    return val

Tasked = df_Prod_final_1[['Encounter #','Disposition Code','Action Code']]
Tasked["Tasked_Status"] = Tasked.apply(T, axis=1)
Tasked = Tasked[Tasked['Tasked_Status'] == "Tasked"][['Encounter #','Tasked_Status']]
df_Prod_final_1 = df_Prod_final_1.merge(Tasked, how= 'left', on="Encounter #")

In [24]:
df_Prod_final_1.to_csv(r"/content/drive/My Drive/data_model_productio.csv", index=False)